In [ ]:
# Spell_Series???

In [ ]:
import numpy as np

import DFS_Functions
import DFS_World_States
from DFS_Functions import adjacent_locations, chebyshev_distance, bresenham_line, calculate_full_path, check_opportunity_attacks, is_line_of_sight_clear, check_visibility, generate_pseudo_history
from DFS_World_States import world, world_grid_states

import DFS_Action_Series

from DFS_Universal_Rules import action_subactions, move_subactions, action_subactions, attack_subactions, free_subactions, subactions_req_targets, object_subactions, target_distance_scores, subactions_req_objects



In [ ]:
# the idea is that if the cast subaction (14) is in the action_series, this function will be called...
# this function will create a number of duplicates of all act_loc_obj_series with the cast subaction,
# and the cast series will be the various spells that the character could cast

In [ ]:
# the cast series may require it's own:
# - location series
# - object series
# - entity series
# ... and that could be a lot...

# so the representation of a spell being cast would look like
        # [[action_series],[location_series],[object_series],[entity_series],[spell_series], etc...] from the outside

# and 
        # [for each spell cast in an action_series: 
        #                       [specific_spell_subclass,
        #                       [spell_options_if_any],         # hardcoded such as shape water or open ended such as suggestion
        #                       [list_of_locations_it_can_target], 
        #                       [list_of_objects_it_can_target], 
        #                       [list_of_entities_it_can_target], 
        #                       ...
        # ] from the inside




In [ ]:
# Spell Series Rules:
# - spell slots
# - casting focus
# - hand requirements
# - sight required
# - spells per turn
# - 

